In [ ]:
# Install nibabel, a Python library for reading and writing medical imaging data
!pip install nibabel

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import nibabel as nib
import tensorflow as tf
from scipy.ndimage import rotate

In [ ]:
# Paths to the CSV files containing postoperative day and clinical characteristics
# Example CSV file is uploaded to the repository
# Replace "PATH1", "PATH2", etc., with actual file paths
train_p_table = pd.read_csv("PATH1")  # Training data (positive screw loosening)
train_n_table = pd.read_csv("PATH2")  # Training data (negative screw loosening)
test_p_table = pd.read_csv("PATH3")  # Test data (positive screw loosening)
test_n_table = pd.read_csv("PATH4")  # Test data (negative screw loosening)

In [ ]:
# Combine all tables to calculate overall mean and standard deviation
combined_data = pd.concat([train_p_table, train_n_table, test_p_table, test_n_table])

# Calculate mean and standard deviation for "Age" and "Postoperative day"
age_mean = combined_data["Age"].mean()
age_std = combined_data["Age"].std()
day_mean = combined_data["Postoperative day"].mean()
day_std = combined_data["Postoperative day"].std()

# Define a function to standardize a column
def standardize_column(table, column, mean, std):
    table[column] = (table[column] - mean) / std  # Standardization formula

# Standardize "Age" and "Postoperative day" in each table
for table in [train_p_table, train_n_table, test_p_table, test_n_table]:
    standardize_column(table, "Age", age_mean, age_std)
    standardize_column(table, "Postoperative day", day_mean, day_std)

In [ ]:
# Paths to the corresponding NIfTI files
train_p_image = "PATH5"  # Training image (positive screw loosening)
train_n_image = "PATH6"  # Training image (negative screw loosening)
test_p_image = "PATH7"  # Test image (positive screw loosening)
test_n_image = "PATH8"  # Test image (negative screw loosening)

In [ ]:
def load_nii_gz_files_to_ndarray(folder_path):
    """
    Load and process all .nii.gz files in a given folder.

    - Reads all NIfTI files in the folder.
    - Loads and rotates each image to maintain consistent orientation.
    - Returns a NumPy array with dimensions (num_files, height, width, depth).
    """
    # Get a sorted list of all NIfTI files in the folder
    nii_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.nii.gz')])

    if not nii_files:
        raise ValueError("No .nii.gz files found in the folder!")

    # Load the first file to determine the image shape
    sample_file = nib.load(os.path.join(folder_path, nii_files[0]))
    x, y, z = sample_file.shape  # Extract dimensions

    # Initialize an empty NumPy array with shape (num_files, y, x, z)
    num_files = len(nii_files)
    rotated_array = np.zeros((num_files, y, x, z), dtype=np.float32)

    # Load and process each file
    for i, file_name in tqdm(enumerate(nii_files)):
        file_path = os.path.join(folder_path, file_name)
        nii_data = nib.load(file_path).get_fdata()  # Convert to NumPy array

        # Rotate 90 degrees clockwise for alignment
        rotated_data = np.rot90(nii_data, k=-1, axes=(0, 1))
        rotated_array[i] = rotated_data

    return rotated_array

In [ ]:
# Convert NIfTI images to NumPy arrays
train_p_image = load_nii_gz_files_to_ndarray(train_p_image)
train_n_image = load_nii_gz_files_to_ndarray(train_n_image)
test_p_image = load_nii_gz_files_to_ndarray(test_p_image)
test_n_image = load_nii_gz_files_to_ndarray(test_n_image)

In [ ]:
# Convert images to float32 for compatibility with TensorFlow models
train_p_image = train_p_image.astype('float32')
train_n_image = train_n_image.astype('float32')
test_p_image = test_p_image.astype('float32')
test_n_image = test_n_image.astype('float32')

In [ ]:
# Create augmented data by flipping along the z-axis
train_p_image = np.concatenate((train_p_image, np.flip(train_p_image, 2)), axis=0)
train_p_image.shape  # Output the new shape

In [ ]:
angles = [-10, 0, 10]  # Rotation angles in degrees

# Augmented data container
augmented_images = []

# Apply rotation to each image
for image in tqdm(train_p_image):
    for angle in angles:
        # Rotate the image along the z-axis (maintains shape)
        rotated_image = rotate(image, angle, axes=(0, 1), reshape=False)
        augmented_images.append(rotated_image)

# Convert list to NumPy array
train_p_image = np.array(augmented_images)
print(f"Augmented data shape: {train_p_image.shape}")

In [ ]:
# Duplicate and extend clinical data to match image augmentation
train_p_table = np.concatenate((train_p_table, np.copy(train_p_table)), axis=0)
train_p_table = np.repeat(train_p_table, repeats=len(angles), axis=0)

print(train_p_table.shape)  # Output the new shape
np.set_printoptions(threshold=np.inf, linewidth=120)  # Print full array

In [ ]:
#Create labels for training and testing Data
label_train_p = np.array([1 for i in range(len(train_p_image))])  # Positive class (screw loosening)
label_train_n = np.array([0 for i in range(len(train_n_image))])  # Negative class (no screw loosening)
label_test_p = np.array([1 for i in range(len(test_p_image))])  # Positive test labels
label_test_n = np.array([0 for i in range(len(test_n_image))])  # Negative test labels

In [ ]:
x_train_val_image = np.concatenate((train_p_image, train_n_image), axis=0)  # Combine training images
x_train_val_table = np.concatenate((train_p_table, train_n_table), axis=0)  # Combine clinical data for training
y_train_val = np.concatenate((label_train_p, label_train_n), axis=0)  # Combine labels for training

x_test_image = np.concatenate((test_p_image, test_n_image), axis=0)  # Combine test images
x_test_table = np.concatenate((test_p_table, test_n_table), axis=0)  # Combine clinical data for testing
y_test = np.concatenate((label_test_p, label_test_n), axis=0)  # Combine labels for testing

In [ ]:
#Expand image dimensions for TensorFlow compatibility
x_train_val_image = tf.expand_dims(x_train_val_image, axis=4)  # Add a channel dimension for training images
x_test_image = tf.expand_dims(x_test_image, axis=4)  # Add a channel dimension for test images

In [ ]:
# Save processed data as a compressed .npz File
np.savez_compressed(
    "PATH9.npz",
    x_train_val_image=x_train_val_image,
    x_train_val_table=x_train_val_table,
    y_train_val=y_train_val,
    x_test_image=x_test_image,
    x_test_table=x_test_table,
    y_test=y_test
)